In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()


real_xtest=xtest
real_ytest=ytest_original

x=xtrain
y=ytrain_original


<IPython.core.display.Javascript object>

In [4]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [5]:

original_scores=[]
trial1_scores=[]
trial2_scores=[]
trial3_scores=[]
trial4_scores=[]
trial5_scores=[]
trial6_scores=[]

from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(x)

print(kf)

data_cv=[]

for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]    
    data_cv.append([[X_train, X_test],[y_train, y_test]])


for i in range(len(data_cv)):
    print('************************** ==> ',i)
    xtrain=data_cv[i][0][0]
    ytrain_original =data_cv[i][1][0] 
    xtest= data_cv[i][0][1]
    ytest_original= data_cv[i][1][1]
    
    
    # xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy() 
    
    # ytrain, ytest = swapcolumns(ytrain, ytest, 2)
    
    
    
    clf=[]
    acc=[]
    finalacc=[]
    ypredproba_all=[]
    ypredconfprob_all=[]
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    original_scores.append(m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    
    #================================================= 
    
    # Class 0
    # ===========================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,0)
    #=================================================
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print(m.f1_score(ytest,rfpred,average='weighted')) 
    
    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest)) 
    
    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=0)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    
    # =================================================
    # classs 1
    # =================================================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain, ytest = swapcolumns(ytrain, ytest, 1)
    # =================================================
    
    rf = RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain, ytrain)
    rfpred = rf.predict(xtest)
    print(m.f1_score(ytest, rfpred,average='weighted'))
    
    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))
    
    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100) 
            
    #=================================================
    # classs 2
    #=================================================
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,2)
    
    #=================================================
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print(m.f1_score(ytest,rfpred,average='weighted'))
    
    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))
    
    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=0)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    # #=================================================
    
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues(acc)
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6433333333333333
    
    trial1_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6783333333333333
    # accuracy_score 0.715
    # accuracy_score 0.7216666666666667
    trial2_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0] * weightvalga[0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1]
    finalcol[:, 1] = ypredproba_all[1][:, 0] * weightvalga[1] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1]
    finalcol[:, 2] = ypredproba_all[2][:, 0] * weightvalga[2] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial3_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1] ,
        -ypredconfprob_all[0][0][1],
        
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        -ypredconfprob_all[1][0][1],
        
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1],
        -ypredconfprob_all[2][0][1],
        
        ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0] + ypredproba_all[0][:, 1]*weightvalga[1]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[2] + ypredproba_all[1][:, 1]*weightvalga[3]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[4] + ypredproba_all[2][:, 1]*weightvalga[5]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    
    trial4_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = weightvalga[0] * (ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = weightvalga[1] * (ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] = weightvalga[2] * (ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial5_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] =  (weightvalga[0] *ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = (weightvalga[1] * ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] =  (weightvalga[2] *ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial6_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    


KFold(n_splits=10, random_state=42, shuffle=True)
************************** ==>  0
original score 0.7848450695747226
0.8080718578254815
0.7930043676910402
0.980233945008309
0
1
2
3
4
5
6
7
8
9
2.451749068875818 [0.98726177 0.85442783 0.99813631]
f1_score 0.7873012878092644
0
1
2
3
4
5
6
7
8
9
1.7797210515820512 [0.97765411 0.93417884 0.99526932]
f1_score 0.7925244427496511
0
1
2
3
4
5
6
7
8
9
1.7954492213295985 [0.97085007 0.9696184  0.99378199]
f1_score 0.7956009864451845
0
1
2
3
4
5
6
7
8
9
1.3332761725113347 [0.96612069 0.72853825 0.96438491 0.72230158 0.9933345  0.89670633]
f1_score 0.4936447606568991
0
1
2
3
4
5
6
7
8
9
2.2640724590904573 [0.9893922  0.89005288 0.99942558]
f1_score 0.7858605404822415
0
1
2
3
4
5
6
7
8
9
2.325013334621122 [0.98417605 0.98023262 0.99996494]
f1_score 0.7906213872033405
************************** ==>  1
original score 0.797176990915303
0.8143508443763926
0.7991059909419634
0.9835020846170539
0
1
2
3
4
5
6
7
8
9
2.491933148624546 [0.96483294 0.9109863

<IPython.core.display.Javascript object>

In [6]:
print("original_scores", np.mean(original_scores), np.std(original_scores))
print("trial1_scores", np.mean(trial1_scores), np.std(trial1_scores))
print("trial2_scores", np.mean(trial2_scores), np.std(trial2_scores))
print("trial3_scores", np.mean(trial3_scores), np.std(trial3_scores))
print("trial2_scores", np.mean(trial4_scores), np.std(trial4_scores))
print("trial3_scores", np.mean(trial5_scores), np.std(trial5_scores))

original_scores 0.7984935640433866 0.006710747704028014
trial1_scores 0.7979369701467055 0.005977307629963453
trial2_scores 0.799340450235737 0.0054975760180904
trial3_scores 0.8013767852014378 0.004766368277410488
trial2_scores 0.4414453945660798 0.13848358254093387
trial3_scores 0.7966861761724666 0.006148977928651287


<IPython.core.display.Javascript object>